In [40]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from pydantic.dataclasses import dataclass
from pydantic import Field
from sympy import integrate, lambdify, oo, symbols, Piecewise
from sympy.core.symbol import Symbol

from binaryStatistics.base_distribution import BaseDistribution

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
@dataclass
class Uniform(BaseDistribution):
    
    dist_parameters: dict = Field(default={})
    
    def __post_init_post_parse__(self):
        super().__init__(self.dist_parameters)
    
    def distribution(self, x, a=None, b=None):
        if a and b:
            self.norm = 1 / (b - a)
        else:
            self.norm = 1
            
        if isinstance(x, Symbol):
            return Piecewise(
                (0, x < a),
                (0, x > b),
                (1 * self.norm, True)
            )
        if isinstance(x, int | float):
            return 1 * self.norm if a <= x <= b else 0.0
        if isinstance(x, np.ndarray):
            return np.piecewise(x, [(x < a), (x <= b) * (x >= a)], [0, 1 * self.norm])

In [46]:
uniform = Uniform(
)

In [47]:
_=plt.hist(uniform.random_sample(-1,50,100000), density=True)

TypeError: '>=' not supported between instances of 'float' and 'NoneType'

In [36]:
uniform.display_cdf()

-Min(0, x) + Min(22, x)

In [37]:
uniform.display_pdf()

Piecewise((0, (x > 22) | (x < 0)), (1, True))

In [38]:
uniform.pdf(np.linspace(-1,0))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [39]:
uniform.cdf(np.linspace(-22,22))

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.04, 0.08, 0.12, 0.16, 0.2 , 0.24, 0.28, 0.32,
       0.36, 0.4 , 0.44, 0.48, 0.52, 0.56, 0.6 , 0.64, 0.68, 0.72, 0.76,
       0.8 , 0.84, 0.88, 0.92, 0.96, 1.  ])